## Loading the libraries

We start by importing the libraries needed to make an EDA on our dataset.

In [2]:
import pandas as pd

## Understanding the dataset

Let us now see what the recipe dataset looks like, what data it offers us.

In [3]:
# Showing the first rows of the dataset

recipe_df = pd.read_csv('../data/raw/RAW_recipes.csv')
recipe_df.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


Let us now separate the nutrition column into the following categories: ‘calories (#)’, ‘total fat (PDV)’, ‘sugar (PDV)’, ‘que sodium (PDV)’, ‘protein (PDV)’, ‘saturated fat (PDV)’, ‘carbohydrates (PDV)’, ‘saturated fat (PDV)’, ‘saturated fat (PDV)’, ‘saturated fat (PDV)’, ‘carbohydrates (PDV)’.

In [4]:
import ast

In [5]:
recipe_df['nutrition'] = recipe_df['nutrition'].apply(ast.literal_eval)
recipe_df['nutrition'] = recipe_df['nutrition'].apply(lambda x: [float(i) for i in x])

In [6]:
nutrition_col_names = ['calories (#)', 'total fat (PDV)', 'sugar (PDV)', 'que sodium (PDV)', 'protein (PDV)', 'saturated fat (PDV)', 'carbohydrates (PDV)']

In [7]:
nutrition_df = recipe_df['nutrition'].apply(pd.Series)
nutrition_df.columns = nutrition_col_names

The columns of this dataframe, with the exception of calories, are expressed in PDV (percentage of recommended daily value), we can transform the values contained in them using absolute units.

In [8]:
nutrition_df.head()

,calories (#),total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV)
0,51.5,0.0,13.0,0.0,2.0,0.0,4.0
1,173.4,18.0,0.0,17.0,22.0,35.0,1.0
2,269.8,22.0,32.0,48.0,39.0,27.0,5.0
3,368.1,17.0,10.0,2.0,14.0,8.0,20.0
4,352.9,1.0,337.0,23.0,3.0,0.0,28.0


For this purpose, we refer to this list of recommended quantities.

In [9]:
transform_values_from_pdv = [
    ('total fat (PDV)', 'total fat (g)', 75),
    ('sugar (PDV)', 'sugar (g)', 100),
    ('sodium (PDV)', 'sodium (mg)', 2300),
    ('protein (PDV)', 'protein (g)', 50),
    ('saturated fat (PDV)', 'saturated fat (g)', 20),
    ('carbohydrates (PDV)', 'carbohydrates (g)', 300)
]

In [10]:
for (old_label, new_label, ratio) in transform_values_from_pdv:
    new_column = nutrition_df[old_label] * ratio / 100
    nutrition_df[new_label] = new_column
    nutrition_df = nutrition_df.drop(columns=[old_label])

In [11]:
nutrition_df.head()

,calories (#),total fat (g),sugar (g),sodium (mg),protein (g),saturated fat (g),carbohydrates (g)
0,51.5,0.00,13.0,0.0,1.0,0.0,12.0
1,173.4,13.50,0.0,391.0,11.0,7.0,3.0
2,269.8,16.50,32.0,1104.0,19.5,5.4,15.0
3,368.1,12.75,10.0,46.0,7.0,1.6,60.0
4,352.9,0.75,337.0,529.0,1.5,0.0,84.0


Once the values have been modified, we add these new columns to the previous dataset.

In [12]:
recipe_df = pd.concat([recipe_df, nutrition_df], axis=1)

recipe_df = recipe_df.drop(columns=['nutrition'])

recipe_df.head()

,name,id,minutes,contributor_id,submitted,tags,n_steps,steps,description,ingredients,n_ingredients,calories (#),total fat (g),sugar (g),sodium (mg),protein (g),saturated fat (g),carbohydrates (g)
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,51.5,0.00,13.0,0.0,1.0,0.0,12.0
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,173.4,13.50,0.0,391.0,11.0,7.0,3.0
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,269.8,16.50,32.0,1104.0,19.5,5.4,15.0
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,368.1,12.75,10.0,46.0,7.0,1.6,60.0
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,352.9,0.75,337.0,529.0,1.5,0.0,84.0


Let's see how many columns we have and the number of entries.

In [13]:
recipe_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231637 entries, 0 to 231636
Data columns (total 18 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   name               231636 non-null  object 
 1   id                 231637 non-null  int64  
 2   minutes            231637 non-null  int64  
 3   contributor_id     231637 non-null  int64  
 4   submitted          231637 non-null  object 
 5   tags               231637 non-null  object 
 6   n_steps            231637 non-null  int64  
 7   steps              231637 non-null  object 
 8   description        226658 non-null  object 
 9   ingredients        231637 non-null  object 
 10  n_ingredients      231637 non-null  int64  
 11  calories (#)       231637 non-null  float64
 12  total fat (g)      231637 non-null  float64
 13  sugar (g)          231637 non-null  float64
 14  sodium (mg)        231637 non-null  float64
 15  protein (g)        231637 non-null  float64
 16  sa

Let's check the number of null values of our df.

In [14]:
recipe_df.isna().sum()

name                    1
id                      0
minutes                 0
contributor_id          0
submitted               0
tags                    0
n_steps                 0
steps                   0
description          4979
ingredients             0
n_ingredients           0
calories (#)            0
total fat (g)           0
sugar (g)               0
sodium (mg)             0
protein (g)             0
saturated fat (g)       0
carbohydrates (g)       0
dtype: int64

There are a handful of null values in the description column, which should be deleted, as this column is relevant when doing NLP with embeddings.

In [15]:
recipe_df.dropna(inplace=True)

We can also remove the id column, which does not contribute anything as far as our recommender is concerned.

In [17]:
recipe_df = recipe_df.drop(columns=['id'])

Let's modify the contrinutor_id column so that instead of the id of the person who published the recipe, the author's relevance or popularity among the published recipes appears.

In [18]:
# Calculating freq of each contrinbutor
recipe_df['contributor_freq'] = recipe_df.groupby('contributor_id')['contributor_id'].transform('count')

In [19]:
recipe_df = recipe_df.drop(columns=['contributor_id'])

Let us now take a look at the most relevant statistics for the numerical variables.

In [20]:
# Descriptive statistics about numeric variables

recipe_df.describe()

,minutes,n_steps,n_ingredients,calories (#),total fat (g),sugar (g),sodium (mg),protein (g),saturated fat (g),carbohydrates (g),contributor_freq
count,2.266570e+05,226657.000000,226657.000000,226657.000000,226657.000000,226657.000000,226657.000000,226657.000000,226657.000000,226657.000000,226657.000000
mean,9.602472e+03,9.773927,9.054170,471.662774,26.952305,83.447059,691.462355,17.300229,9.072963,46.352255,289.277874
std,4.510715e+06,5.999824,3.740942,1194.845397,58.300010,807.100380,3032.851012,29.241828,19.527666,247.448563,547.371364
min,0.000000e+00,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,2.000000e+01,6.000000,6.000000,174.500000,6.000000,9.000000,115.000000,3.500000,1.400000,12.000000,12.000000
50%,4.000000e+01,9.000000,9.000000,313.300000,15.000000,25.000000,345.000000,9.000000,4.600000,27.000000,79.000000
75%,6.500000e+01,12.000000,11.000000,518.700000,30.750000,68.000000,759.000000,25.500000,10.400000,48.000000,313.000000
max,2.147484e+09,145.000000,43.000000,434360.200000,12887.250000,362729.000000,674774.000000,3276.000000,2079.000000,108294.000000,3118.000000


With this function we will calculate the number of outliers that these numerical columns have, using as a basis statistics such as Q1, Q3 and IQR.

In [21]:
def outlier_table(df):
    numeric_recipe_df = df.select_dtypes(include='number')

    Q1 = numeric_recipe_df.quantile(0.25)
    Q2 = numeric_recipe_df.quantile(0.5)
    Q3 = numeric_recipe_df.quantile(0.75)
    IQR = Q3 - Q1

    lower_bound_for_outliers = Q1 - 1.5 * IQR
    upper_bound_for_outliers = Q3 + 1.5 * IQR

    outliers = numeric_recipe_df.apply(lambda x: (x < lower_bound_for_outliers[x.name]) | (x > upper_bound_for_outliers[x.name]))
    outliers_count = outliers.sum()
    proportion_outliers = outliers_count / len(numeric_recipe_df)

    results = pd.DataFrame({
        'Lower bound for outliers': lower_bound_for_outliers,
        'Q1': Q1,
        'Median/Q2': Q2,
        'Q3': Q3,
        'Upper bound for outliers': upper_bound_for_outliers,
        'IQR': IQR,
        'Proportion of Outliers': proportion_outliers,
        'Rows Affected': outliers_count
    })

    print("IQR and outlier proportion table:")

    return results

Below we see the results of the table, where we also see the proportions of outliers over the total number of elements in the df.

In [22]:
outliers = outlier_table(recipe_df)
outliers

IQR and outlier proportion table:


,Lower bound for outliers,Q1,Median/Q2,Q3,Upper bound for outliers,IQR,Proportion of Outliers,Rows Affected
minutes,-47.500,20.0,40.0,65.00,132.500,45.00,0.102684,23274
n_steps,-3.000,6.0,9.0,12.00,21.000,6.00,0.043400,9837
n_ingredients,-1.500,6.0,9.0,11.00,18.500,5.00,0.016386,3714
calories (#),-341.800,174.5,313.3,518.70,1035.000,344.20,0.067384,15273
total fat (g),-31.125,6.0,15.0,30.75,67.875,24.75,0.069550,15764
sugar (g),-79.500,9.0,25.0,68.00,156.500,59.00,0.101890,23094
sodium (mg),-851.000,115.0,345.0,759.00,1725.000,644.00,0.068372,15497
protein (g),-29.500,3.5,9.0,25.50,58.500,22.00,0.039169,8878
saturated fat (g),-12.100,1.4,4.6,10.40,23.900,9.00,0.070477,15974
carbohydrates (g),-42.000,12.0,27.0,48.00,102.000,36.00,0.066228,15011


In [23]:
threshold = outliers.drop(index='contributor_freq')
threshold

,Lower bound for outliers,Q1,Median/Q2,Q3,Upper bound for outliers,IQR,Proportion of Outliers,Rows Affected
minutes,-47.500,20.0,40.0,65.00,132.500,45.00,0.102684,23274
n_steps,-3.000,6.0,9.0,12.00,21.000,6.00,0.043400,9837
n_ingredients,-1.500,6.0,9.0,11.00,18.500,5.00,0.016386,3714
calories (#),-341.800,174.5,313.3,518.70,1035.000,344.20,0.067384,15273
total fat (g),-31.125,6.0,15.0,30.75,67.875,24.75,0.069550,15764
sugar (g),-79.500,9.0,25.0,68.00,156.500,59.00,0.101890,23094
sodium (mg),-851.000,115.0,345.0,759.00,1725.000,644.00,0.068372,15497
protein (g),-29.500,3.5,9.0,25.50,58.500,22.00,0.039169,8878
saturated fat (g),-12.100,1.4,4.6,10.40,23.900,9.00,0.070477,15974
carbohydrates (g),-42.000,12.0,27.0,48.00,102.000,36.00,0.066228,15011


We remove the frequency of publication of recipes and proceed to delete all outliers.

In [24]:
clean_recipe = recipe_df

for index, row in threshold.iterrows():
    column = index
    lbo = row['Lower bound for outliers']
    ubo = row['Upper bound for outliers']

    clean_recipe = clean_recipe[(clean_recipe[column] >= lbo) & (clean_recipe[column] <= ubo)]


After performing the action our df has the following information.

In [25]:
clean_recipe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 156211 entries, 0 to 231636
Data columns (total 17 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   name               156211 non-null  object 
 1   minutes            156211 non-null  int64  
 2   submitted          156211 non-null  object 
 3   tags               156211 non-null  object 
 4   n_steps            156211 non-null  int64  
 5   steps              156211 non-null  object 
 6   description        156211 non-null  object 
 7   ingredients        156211 non-null  object 
 8   n_ingredients      156211 non-null  int64  
 9   calories (#)       156211 non-null  float64
 10  total fat (g)      156211 non-null  float64
 11  sugar (g)          156211 non-null  float64
 12  sodium (mg)        156211 non-null  float64
 13  protein (g)        156211 non-null  float64
 14  saturated fat (g)  156211 non-null  float64
 15  carbohydrates (g)  156211 non-null  float64
 16  contrib

In [26]:
clean_recipe.describe()

,minutes,n_steps,n_ingredients,calories (#),total fat (g),sugar (g),sodium (mg),protein (g),saturated fat (g),carbohydrates (g),contributor_freq
count,156211.000000,156211.000000,156211.000000,156211.000000,156211.000000,156211.000000,156211.000000,156211.000000,156211.000000,156211.000000,156211.000000
mean,37.598287,8.728604,8.635167,280.277298,15.477671,33.018795,384.112854,12.323028,5.073822,25.343932,280.196203
std,25.598449,4.308051,3.301332,174.949715,13.294340,35.670242,378.104543,12.639379,5.074225,20.481792,522.602151
min,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,20.000000,6.000000,6.000000,144.700000,5.250000,8.000000,92.000000,2.500000,1.200000,9.000000,12.000000
50%,30.000000,8.000000,8.000000,250.700000,12.000000,19.000000,253.000000,7.000000,3.400000,21.000000,78.000000
75%,50.000000,11.000000,11.000000,385.300000,22.500000,46.000000,575.000000,19.500000,7.600000,36.000000,309.000000
max,132.000000,21.000000,18.000000,1034.800000,67.500000,156.000000,1725.000000,58.500000,23.800000,102.000000,3118.000000


We will also modify the minimum values of our dataset to match realistic data regarding the preparation of a recipe.

In [28]:
reasonable_value_range = {
    'minutes': (5, 300),
    'n_steps': (1, 50),
    'n_ingredients': (2, 50),
    'calories (#)': (10, 2000), # cals
    'total fat (g)': (0, 400), # g
    'sugar (g)': (0, 300), # g
    'sodium (mg)': (0, 5000), # mg
    'protein (g)': (0, 400), # g
    'saturated fat (g)': (0, 200), # g
    'carbohydrates (g)' : (0, 400) # g
}

In [29]:
for col in reasonable_value_range.keys():
    clean_recipe = clean_recipe[clean_recipe[col] >= reasonable_value_range[col][0]]

After applying the changes, the size of the dataset is as follows.

In [30]:
len(clean_recipe)

151489

In [31]:
clean_recipe.describe()

,minutes,n_steps,n_ingredients,calories (#),total fat (g),sugar (g),sodium (mg),protein (g),saturated fat (g),carbohydrates (g),contributor_freq
count,151489.000000,151489.000000,151489.000000,151489.000000,151489.000000,151489.000000,151489.000000,151489.000000,151489.000000,151489.000000,151489.000000
mean,38.596677,8.866525,8.750378,284.587672,15.795365,33.113942,392.191077,12.605364,5.173288,25.724383,280.715788
std,25.261037,4.265882,3.260251,174.124496,13.262481,35.594877,378.497365,12.680439,5.082608,20.476757,525.906150
min,5.000000,1.000000,2.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,20.000000,6.000000,6.000000,149.200000,5.250000,8.000000,92.000000,3.000000,1.200000,9.000000,12.000000
50%,33.000000,8.000000,9.000000,255.200000,12.750000,19.000000,276.000000,7.500000,3.600000,21.000000,76.000000
75%,50.000000,11.000000,11.000000,389.100000,23.250000,46.000000,598.000000,20.000000,7.600000,36.000000,309.000000
max,132.000000,21.000000,18.000000,1034.800000,67.500000,156.000000,1725.000000,58.500000,23.800000,102.000000,3118.000000


Finally, we save the updated data.

In [32]:
clean_recipe.to_csv("../data/processed/recipe_to_db.csv", index=False)